In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

In [4]:
foreign_investment = pd.read_csv('FORINV.csv')
gini_index = pd.read_csv('GINI.csv')
human_capital_index = pd.read_csv('HCI.csv')
inflation = pd.read_csv('INFLATION.csv')
interest_rates = pd.read_csv('INTRATE.csv')
highest_income_tax_bracket = pd.read_csv('ITAX.csv')
poverty_rate = pd.read_csv('POVERTY.csv')
real_consumer_spending = pd.read_csv('RCONSPEND.csv')
real_GDP = pd.read_csv('RGDP.csv')
real_median_weekly_earnings = pd.read_csv('RMEDEARN.csv')
real_med_household_income = pd.read_csv('RMEDHOUINC.csv')
unemployment_rate = pd.read_csv('UNRATE.csv')